In [ ]:
%pip install pyspark

In [ ]:
%pip install tensorflow --ignore-installed --user

In [ ]:
import pyspark

In [ ]:
# import cv2

# # Load the image into a NumPy array
# image_path = "C:/Users/Saad/OneDrive - Institute of Business Administration/IBA code/6th Semester\Parralel computing/RoadDetection/testing_data/verypoor_test.jpg"
# image_np = cv2.imread(image_path)

# # Resize the image to the expected input size of the TensorFlow model
# input_size = (224, 224)
# image_np_resized = cv2.resize(image_np, input_size)

#Use this to get image preprocessed

import cv2
import numpy as np

# Load the test image into a NumPy array
image_path = "C:/Users/Saad/OneDrive - Institute of Business Administration/IBA code/6th Semester\Parralel computing/RoadDetection/testing_data/verypoor_test.jpg"
image_np = cv2.imread(image_path)

# Resize the image to the expected input size of the TensorFlow model
input_size = (224, 224)
image_np_resized = cv2.resize(image_np, input_size)

# Scale the pixel values to the range [0, 1]
image_np_resized = image_np_resized / 255.0

# Add a batch dimension to the image array
image_np_resized = np.expand_dims(image_np_resized, axis=0)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

# Create a SparkSession object
spark = SparkSession.builder.appName("RoadPrediction").getOrCreate()

# Convert the resized image to a vector
image_vec = Vectors.dense(image_np_resized.flatten())

# Create a PySpark DataFrame containing a single row with the image vector
image_df = spark.createDataFrame([(image_vec,)], ["features"])

In [ ]:
%pip install tensorflow

In [ ]:
%pip install sparkdl

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
from pyspark.ml import PipelineModel
import sparkdl as sdl

In [ ]:
from sparkdl import TFModel

# Specify the paths to the model files and the input/output node names
model_path = "C:/Users/Saad/OneDrive - Institute of Business Administration/IBA code/6th Semester/Parralel computing/RoadDetection/saved_model/saved_model.pb"
input_node_name = "input_features"
output_node_name = "output_scores"

# Load the TensorFlow model in PySpark
tf_model = TFModel(modelPath=model_path,
                   inputCol=input_node_name,
                   outputCol=output_node_name)

In [ ]:
# import numpy as np
# from pyspark.sql import SparkSession
# from pyspark.ml.linalg import Vectors

# # Create a SparkSession object
# spark = SparkSession.builder.appName("MyApp").getOrCreate()

# def predict_profile(image_path, model):
#     print("Predicting .................................")
#     ar = convert_to_array(image_path)
#     ar = ar / 255
#     image_vec = Vectors.dense(ar.flatten())
#     image_df = spark.createDataFrame([(image_vec,)], ["features"])
#     predictions = model.transform(image_df)
#     label_index = np.argmax(predictions.collect()[0]['output_scores'])
#     profile = get_profile_name(label_index)
#     acc = predictions.collect()[0]['output_scores'][label_index]
#     print("The predicted profile is a " + profile + " with accuracy = " + str(acc))
#     return predictions

# # Example usage:
# model_path = "/path/to/extracted/model"
# input_node_name = "input_features"
# output_node_name = "output_scores"
# model = load_pretrained_model(model_path, input_node_name, output_node_name)
# test_image_path = "/path/to/test_image.jpg"
# predictions = predict_profile(test_image_path, model)
# predictions.show()

In [ ]:
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import udf
from tensorflow import keras
from tensorflow.compat.v1.saved_model import load as tf_load_model
import tensorflow.compat.v1 as tf
import numpy as np

In [ ]:
# # Load the TensorFlow Keras pretrained model in the form of a .pb file:
# tf_model = tf.saved_model.load("C:/Users/Saad/OneDrive - Institute of Business Administration/IBA code/6th Semester/Parralel computing/RoadDetection/saved_model",  tags=["serve"], export_dir='')
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], "C:/Users/Saad/OneDrive - Institute of Business Administration/IBA code/6th Semester/Parralel computing/RoadDetection/saved_model")
    

In [ ]:
def predict_fn(x):
    input_tensor = tf_model.signatures["serving_default"].inputs["input_1"]
    output_tensor = tf_model.signatures["serving_default"].outputs["output_1"]
    prediction = tf_model(input_tensor=tf.cast(x, dtype=tf.float32))[output_tensor].numpy()
    return np.argmax(prediction)

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, ArrayType

# Define a UDF that applies the pre-trained model to each row of the DataFrame
predict_udf = udf(predict_fn, IntegerType())

from pyspark.sql.types import DoubleType, ArrayType

# Define the element type of the array
element_type = DoubleType()

# Create an ArrayType with the specified element type
array_type = ArrayType(element_type)

# Apply the UDF to the test DataFrame to get the predicted label for each image
predicted_df = image_df.withColumn("predicted_label", predict_udf(image_df["features"].cast(array_type)))